In [ ]:
import os
import yaml
import pandas as pd
from pyathena import connect
from pyathena.util import as_pandas
from sqlalchemy import *
from datetime import datetime

from iefp import processing

%load_ext autoreload

In [ ]:
pg_cred = yaml.load(open("../conf/local/credentials.yml"), Loader=yaml.FullLoader)

In [ ]:
url = 'postgresql://{}:{}@{}:{}/{}'
url = url.format(pg_cred["pg_user"], pg_cred["pg_pass"], pg_cred["pg_host"], 5432, "iefp")

# The return value of create_engine() is our connection object
con = create_engine(url, client_encoding='utf8')

# We then bind the connection to MetaData()
meta = MetaData(bind=con, reflect=True)

In [ ]:
int_cols = yaml.load(open("../conf/base/sigae_columns.yml"), Loader=yaml.FullLoader)["intervencoes"]
table = 'intervencoes'

In [ ]:
sql = """
select {}
from {}
order by ano_mes
limit 100000
""".format(', '.join(int_cols), "intervencoes")

df = pd.read_sql(sql, con)

## Test Functions

In [ ]:
cleaned = processing.clean_string(df.f_dcurso)
cleaned.drop_duplicates().to_list()

In [ ]:
# still some rouge brackets and -

In [ ]:
date_output = processing.object_to_date(df.data_resultado,"%Y-%m-%d %H:%M:%S")
date_output.head(100)